In [1]:
import numpy as np
import pandas as pd
import robin_stocks as r
import inspect
import math
import matplotlib.pyplot as plt
import datetime as dt

# Trade options using RL - https://towardsdatascience.com/deep-reinforcement-learning-for-automated-stock-trading-f1dad0126a02

Also use robintrack for popularity data
## State space definition
Select 10 options around the atm price  
1. cash - money held as cash - 1  
2. price - bid prices of options we can trade - n    
    3. vegas - list of vega for each option - n  
    4. gammma - list of gammas for each option - n   
    5. deltas - list of deltas for each option - n  
    6. thetas - list of thetas for each option - n  
    7. ivs - list of ivs for each option - n
3.  how many days to earning - 1  
4. for each options how many days from earnings is the option expiration date - n  
8. spy - spy price - 1 
9. stock - price of share - 1   
10. position - for each option are we -1,0 or 1 - n  
11. cost - for each option what was our purchase price - n 
12. historical price of option - array of price of options for last 7 days - 7n  
13. historical price of stock - array of price of stock for last 7 days - 7n  

## Action space definition
a - buy/sell/hold - for each option either -1,0,1 - n  
  
## Reward function
r(s,a,s') - cash + price * position

## Policy
p(s) - probability distribution of actions to take at s

## Q-value
Q(a,s) - expected value of taking action a at state s

In [5]:
import robin_stocks as r
login = r.login('anshul.tibrewal2203@gmail.com', "Icangeta45!")

In [15]:
#!!! fill out the specific option information
symbol = 'AAPL'
symbol_name = r.get_name_by_symbol(symbol)
expirationDate = '2020-10-09' # format is YYYY-MM-DD.
strike = 300
optionType = 'call' # available options are 'call' or 'put' or None.
interval = '10minute' # available options are '5minute', '10minute', 'hour', 'day', and 'week'.
span = 'week' # available options are 'day', 'week', 'year', and '5year'.
bounds = 'regular' # available options are 'regular', 'trading', and 'extended'.
info = None
#!!!
r.get_chains(symbol)

{'id': '7dd906e5-7d4b-4161-a3fe-2c3b62038482',
 'symbol': 'AAPL',
 'can_open_position': True,
 'cash_component': None,
 'expiration_dates': ['2020-10-02',
  '2020-10-09',
  '2020-10-16',
  '2020-10-23',
  '2020-10-30',
  '2020-11-06',
  '2020-11-20',
  '2020-12-18',
  '2021-01-15',
  '2021-03-19',
  '2021-04-16',
  '2021-06-18',
  '2021-09-17',
  '2022-01-21',
  '2022-06-17',
  '2022-09-16',
  '2023-01-20'],
 'trade_value_multiplier': '100.0000',
 'underlying_instruments': [{'id': '3b1b2528-8887-4410-bce4-b5128eac4a86',
   'instrument': 'https://api.robinhood.com/instruments/450dfc6d-5510-4d40-abfb-f633b7d9be3e/',
   'quantity': 100}],
 'min_ticks': {'above_tick': '0.05',
  'below_tick': '0.01',
  'cutoff_price': '3.00'}}

In [19]:
# r.get_open_option_positions()

In [16]:
historicalData = r.get_option_historicals(symbol, expirationDate, strike, optionType, interval, span, bounds)

dates = []
closingPrices = []
openPrices = []

for data_point in historicalData:
    dates.append(data_point['begins_at'])
    closingPrices.append(data_point['close_price'])
    openPrices.append(data_point['open_price'])

# change the dates into a format that matplotlib can recognize.
x = [dt.datetime.strptime(d,'%Y-%m-%dT%H:%M:%SZ') for d in dates]

Getting the option ID failed. Perhaps the expiration date is wrong format, or the strike price is wrong.
404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/historicals/None/?span=week&interval=10minute&bounds=regular


TypeError: 'NoneType' object is not iterable

## Get all stock data we need for state space

In [19]:
def get_stock_state(ticker="AAPL"):
    stock_prices = r.get_stock_historicals(ticker, interval='day', span='3month', bounds='regular')
    earnings = r.stocks.get_earnings(ticker)
    return {"P" : stock_prices, "e" : earnings}

In [20]:
get_stock_state()

AttributeError: module 'robin_stocks' has no attribute 'get_stock_historicals'

In [50]:
!!! fill out the specific option information
symbol = 'AAPL'
expirationDate = '2020-09-25'
strike = 200
optionType = 'call'
span = 'day' #available options are day,week,year, and 5year
#!!!
days = 20
chain = r.options.get_chains(symbol)
tradeable = r.options.find_tradable_options(symbol = symbol, expirationDate = chain["expiration_dates"][0])
historicalData = r.get_option_historicals(symbol,expirationDate,strike,optionType,span)

dates = []
closingPrices = []
openPrices = []

for item in historicalData:
#     print(item)
    dates.append(dt.datetime.strptime(item['begins_at'],'%Y-%m-%dT%H:%M:%SZ'))
    closingPrices.append(float(item['close_price']))
    openPrices.append(float(item['open_price']))

# print(openPrices)
#!!! I made it so it only got the last 30 days but you could delete these lines to get full year.
dates = dates[-days:]
closingPrices = closingPrices[-days:]
openPrices = openPrices[-days:]
#
start_date = dates[0]


# change the dates into a format that matplotlib can recognize.
# x = [dt.datetime.strptime(d,'%Y-%m-%dT%H:%M:%SZ') for d in dates]
x = []
for d in dates:
    seconds = (d - start_date).total_seconds()/86400
    x.append(seconds)
    


plt.figure(figsize=(5,4))
# plots the data.
plt.plot(x, closingPrices, 'ro')
# plt.plot(x, openPrices, 'bo')
plt.title("Option price for "+r.get_name_by_symbol(symbol)+" over time")
plt.xlabel("Dates")
plt.ylabel("Price")
plt.show()

Found Additional pages.
Loading page 2 ...
Loading page 3 ...
Loading page 4 ...
Loading page 5 ...
Loading page 6 ...
Loading page 7 ...
Loading page 8 ...
Loading page 9 ...
Loading page 10 ...
Loading page 11 ...
Loading page 12 ...
Loading page 13 ...
Loading page 14 ...
Loading page 15 ...
Loading page 16 ...
Loading page 17 ...
Loading page 18 ...
Loading page 19 ...
Loading page 20 ...
Loading page 21 ...
Loading page 22 ...
Loading page 23 ...
Loading page 24 ...
Loading page 25 ...
Loading page 26 ...
Loading page 27 ...
Getting the option ID failed. Perhaps the expiration date is wrong format, or the strike price is wrong.
404 Client Error: Not Found for url: https://api.robinhood.com/marketdata/options/historicals/None/?span=week&interval=day&bounds=regular


TypeError: 'NoneType' object is not iterable

In [34]:
print(historicalData[0])

{'begins_at': '2020-09-14T13:30:00Z', 'open_price': '20.430000', 'close_price': '20.430000', 'high_price': '20.430000', 'low_price': '20.430000', 'volume': 0, 'session': 'reg', 'interpolated': False, 'symbol': 'ENPH'}


## Implementing Soft Actor Critic in this notebook

In [4]:
import math
import random

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display

%matplotlib inline

use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

## Auxillary functions

In [6]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0
    
    def push(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = map(np.stack, zip(*batch))
        return state, action, reward, next_state, done
    
    def __len__(self):
        return len(self.buffer)

In [9]:
class NormalizedActions():
    def _action(self, action):
        low  = -1
        high = 1
        
        action = low + (action + 1.0) * 0.5 * (high - low)
        action = np.clip(action, low, high)
        
        return action

    def _reverse_action(self, action):
        low  = -1
        high = 1
        
        action = 2 * (action - low) / (high - low) - 1
        action = np.clip(action, low, high)
        return action


## Define Networks

In [10]:
class ValueNetwork(nn.Module):
    def __init__(self, state_dim, hidden_dim, init_w=3e-3):
        super(ValueNetwork, self).__init__()
        
        self.linear1 = nn.Linear(state_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, hidden_dim)
        self.linear3 = nn.Linear(hidden_dim, 1)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
        
        
class SoftQNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3):
        super(SoftQNetwork, self).__init__()
        
        self.linear1 = nn.Linear(num_inputs + num_actions, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        self.linear3 = nn.Linear(hidden_size, 1)
        
        self.linear3.weight.data.uniform_(-init_w, init_w)
        self.linear3.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state, action):
        x = torch.cat([state, action], 1)
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return x
        
        
class PolicyNetwork(nn.Module):
    def __init__(self, num_inputs, num_actions, hidden_size, init_w=3e-3, log_std_min=-20, log_std_max=2):
        super(PolicyNetwork, self).__init__()
        
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
        
        self.linear1 = nn.Linear(num_inputs, hidden_size)
        self.linear2 = nn.Linear(hidden_size, hidden_size)
        
        self.mean_linear = nn.Linear(hidden_size, num_actions)
        self.mean_linear.weight.data.uniform_(-init_w, init_w)
        self.mean_linear.bias.data.uniform_(-init_w, init_w)
        
        self.log_std_linear = nn.Linear(hidden_size, num_actions)
        self.log_std_linear.weight.data.uniform_(-init_w, init_w)
        self.log_std_linear.bias.data.uniform_(-init_w, init_w)
        
    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        
        mean    = self.mean_linear(x)
        log_std = self.log_std_linear(x)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        
        return mean, log_std
    
    def evaluate(self, state, epsilon=1e-6):
        mean, log_std = self.forward(state)
        std = log_std.exp()
        
        normal = Normal(0, 1)
        z      = normal.sample()
        action = torch.tanh(mean+ std*z.to(device))
        log_prob = Normal(mean, std).log_prob(mean+ std*z.to(device)) - torch.log(1 - action.pow(2) + epsilon)
        return action, log_prob, z, mean, log_std
        
    
    def get_action(self, state):
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        mean, log_std = self.forward(state)
        std = log_std.exp()
        
        normal = Normal(0, 1)
        z      = normal.sample().to(device)
        action = torch.tanh(mean + std*z)
        
        action  = action.cpu()#.detach().cpu().numpy()
        return action[0]

## Update functions

In [11]:
def update(batch_size,gamma=0.99,soft_tau=1e-2,):
    
    state, action, reward, next_state, done = replay_buffer.sample(batch_size)

    state      = torch.FloatTensor(state).to(device)
    next_state = torch.FloatTensor(next_state).to(device)
    action     = torch.FloatTensor(action).to(device)
    reward     = torch.FloatTensor(reward).unsqueeze(1).to(device)
    done       = torch.FloatTensor(np.float32(done)).unsqueeze(1).to(device)

    predicted_q_value1 = soft_q_net1(state, action)
    predicted_q_value2 = soft_q_net2(state, action)
    predicted_value    = value_net(state)
    new_action, log_prob, epsilon, mean, log_std = policy_net.evaluate(state)

    
    
# Training Q Function
    target_value = target_value_net(next_state)
    target_q_value = reward + (1 - done) * gamma * target_value
    q_value_loss1 = soft_q_criterion1(predicted_q_value1, target_q_value.detach())
    q_value_loss2 = soft_q_criterion2(predicted_q_value2, target_q_value.detach())


    soft_q_optimizer1.zero_grad()
    q_value_loss1.backward()
    soft_q_optimizer1.step()
    soft_q_optimizer2.zero_grad()
    q_value_loss2.backward()
    soft_q_optimizer2.step()    
# Training Value Function
    predicted_new_q_value = torch.min(soft_q_net1(state, new_action),soft_q_net2(state, new_action))
    target_value_func = predicted_new_q_value - log_prob
    value_loss = value_criterion(predicted_value, target_value_func.detach())

    
    value_optimizer.zero_grad()
    value_loss.backward()
    value_optimizer.step()
# Training Policy Function
    policy_loss = (log_prob - predicted_new_q_value).mean()

    policy_optimizer.zero_grad()
    policy_loss.backward()
    policy_optimizer.step()
    
    
    for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - soft_tau) + param.data * soft_tau
        )

## Initializations

In [12]:
env = NormalizedActions()

action_dim = env.action_space.shape[0]
state_dim  = env.observation_space.shape[0]
hidden_dim = 256

value_net        = ValueNetwork(state_dim, hidden_dim).to(device)
target_value_net = ValueNetwork(state_dim, hidden_dim).to(device)

soft_q_net1 = SoftQNetwork(state_dim, action_dim, hidden_dim).to(device)
soft_q_net2 = SoftQNetwork(state_dim, action_dim, hidden_dim).to(device)
policy_net = PolicyNetwork(state_dim, action_dim, hidden_dim).to(device)

for target_param, param in zip(target_value_net.parameters(), value_net.parameters()):
    target_param.data.copy_(param.data)
    

value_criterion  = nn.MSELoss()
soft_q_criterion1 = nn.MSELoss()
soft_q_criterion2 = nn.MSELoss()

value_lr  = 3e-4
soft_q_lr = 3e-4
policy_lr = 3e-4

value_optimizer  = optim.Adam(value_net.parameters(), lr=value_lr)
soft_q_optimizer1 = optim.Adam(soft_q_net1.parameters(), lr=soft_q_lr)
soft_q_optimizer2 = optim.Adam(soft_q_net2.parameters(), lr=soft_q_lr)
policy_optimizer = optim.Adam(policy_net.parameters(), lr=policy_lr)


replay_buffer_size = 1000000
replay_buffer = ReplayBuffer(replay_buffer_size)

NameError: name 'gym' is not defined

## Training loop

In [14]:
while frame_idx < max_frames:
    state = env.reset()
    episode_reward = 0
    
    for step in range(max_steps):
        if frame_idx >1000:
            action = policy_net.get_action(state).detach()
            next_state, reward, done, _ = env.step(action.numpy())
        else:
            action = env.action_space.sample()
            next_state, reward, done, _ = env.step(action)
        
        
        replay_buffer.push(state, action, reward, next_state, done)
        
        state = next_state
        episode_reward += reward
        frame_idx += 1
        
        if len(replay_buffer) > batch_size:
            update(batch_size)
        
        if frame_idx % 1000 == 0:
            plot(frame_idx, rewards)
        
        if done:
            break
        
    rewards.append(episode_reward)



NameError: name 'frame_idx' is not defined